# Importing and defining constants

In [ ]:
# -*- coding: cp1252 -*-
import sys
import serial
import codecs
import re
import serial.tools.list_ports

from PyQt4.QtGui import *
from PyQt4.QtCore import *

NIR_RANGE_MAX_BOUNDARY_MHZ = 3072       # Near Inrared frequency channel
NIR_RANGE_MIN_BOUNDARY_MHZ = 1472

UV_RANGE_MAX_BOUNDARY_MHZ = 2080
UV_RANGE_MIN_BOUNDARY_MHZ = 1040

VIS_RANGE_MAX_BOUNDARY_MHZ = 2496       # Visible light frequency channel
VIS_RANGE_MIN_BOUNDARY_MHZ = 1152

RF_REF_MIN_MHZ = 20
RF_REF_MAX_MHZ = 250

RF_PFD_MIN_KHZ = 40
RF_PFD_MAX_KHZ = 100000

# Function verifying the serial port's usability

In [ ]:
def port_is_usable(port):
        try:  # Opening test of the serial port
                ser=serial.Serial(port)
                return True
        except serial.serialutil.SerialException:  # If the test fails, the following command is executed
                return False

# Fonction verifying a text file's readability

In [ ]:
def file_is_readable(txtFile):
        try: # Opening test of the specified file
                open(txtFile,"r")  # "r" = read
                return True
        except IOError:
                return False

# Function sending instructions through the serial port

In [ ]:
def send_instructions(iLatch, fLatch, rLatch, abLatch, port):
    # The "with...as..." loop allows to open and then close properly the serial port used
    if port_is_usable(port)==True:  # Check if the port is usable
            with serial.Serial(port) as ser :
                iLatch_toSend=iLatch.decode('hex')  # Converts the hexa string into a series of bytes
                fLatch_toSend=fLatch.decode('hex')  # with 2 alphanumeric values per byte
                rLatch_toSend=rLatch.decode('hex')
                abLatch_toSend=abLatch.decode('hex')
                
                ser.write(iLatch_toSend)            # Data sent through the serial port
                compteur=7
                while(compteur>0):                  # Temporization loop to ensure the previous data had
                      compteur=compteur-1           # time to be sent
                ser.write(fLatch_toSend)
                compteur=7
                while(compteur>0):
                      compteur=compteur-1
                ser.write(rLatch_toSend)
                compteur=7
                while(compteur>0):
                      compteur=compteur-1
                ser.write(abLatch_toSend)
                
    else:  # If the port is unusable, a message is addressed to the user
            QMessageBox.about(window,'Serial port error','Please check that if device is connected to the right port')
            return

# Class organizing the main window

## Designing of the window

In [ ]:
class MainWindow(QWidget):
    def __init__(self):
        QWidget.__init__(self)
        self.setWindowTitle("PLL Controller version 0.1")
        self.resize(800, 600)
        self.move(20,0)  # The window will be shown at this position of the screen
        self.setWindowIcon(QIcon("logo3.png"))  # Assignation of a window icon

## Defining the lists containing the items to add to the ComboBoxes (drop down lists)

In [ ]:
        self.serialPorts=[]
        for port in serial.tools.list_ports.comports():  # Browse of the list of serial ports to which a device is connected
                if port[2] != 'n/a':
                        port_str=str(port[:2])
                        self.serialPorts.append(port_str.rstrip('\''))  # Add ports to our list
        self.channels=['Near-infrared channel (NIR) - 1472-3072 MHz', 'UV channel - 1040-2080 MHz',\
                       'Visible light channel (VIS) - 1152-2496 MHz']
        self.prescalerValues=['8/9', '16/17', '32/33', '64/65']
        self.CPsetting=['0,625 mA', '1,25 mA', '1,875 mA', '2,5 mA', '3,125 mA', '3,75 mA', '4,375 mA', '5,0 mA']
        self.CPgain=['0', '1']
        self.CPoutput=['Normal','Three-state']
        self.fastlockMode=['Disabled', 'Mode 1', 'Mode 2']
        self.timeoutCycles=['3', '7', '11', '15', '19', '23', '27', '31', '35', '39', '43', '47',
                            '51', '55', '59', '63']
        self.PFDpolarity=['Negative', 'Positive']
        self.counterReset=['Disabled', 'Enabled']
        self.lockDetectPrecision=['3 cycles', '5 cycles']
        self.powerDown=['Normal operation', 'Asynchronous power-down', 'Synchronous power-down']
        self.abpw=['2,9 ns', '1,3 ns TEST MODE ONLY', '6,0 ns', '2,9 ns']
        self.muxout=['Three-state output', 'Digital lock detect', 'N divider output', 'DVDD',
                     'R divider output', 'Analog lock detect', 'Serial data output', 'DGND']

## Constitution of the LineEdits

In [ ]:
        self.rfref_line=QLineEdit()
        self.rfpfd_line=QLineEdit()
        self.spacingTab1_line=QLineEdit()
        self.startFreq_line=QLineEdit()
        self.startFreq_line.setFixedWidth(50)
        self.stopFreq_line=QLineEdit()
        self.stopFreq_line.setFixedWidth(50)
        self.spacingTab2_line=QLineEdit()
        self.spacingTab2_line.setFixedWidth(50)
        self.timeDelay_line=QLineEdit()
        self.timeDelay_line.setFixedWidth(50)

## Constitution of the PushButtons

In [ ]:
        self.writePLL_button=QPushButton("Write PLL")
        self.writePLL_button.setFixedWidth(90)
        self.writePLL_button.setFixedHeight(35)
        self.autoSweep_button=QPushButton("Auto Sweep")
        self.autoSweep_button.setFixedWidth(90)
        self.autoSweep_button.setFixedHeight(35)
        self.stop_buttonTab1=QPushButton("Stop / Reset")
        self.stop_buttonTab1.setFixedWidth(90)
        self.stop_buttonTab1.setFixedHeight(35)
        self.quit_buttonTab1=QPushButton("Quit")
        self.quit_buttonTab1.setFixedWidth(50)
        self.quit_buttonTab1.setFixedHeight(35)
        self.load_buttonTab1=QPushButton("Load settings")
        self.load_buttonTab1.setFixedWidth(100)
        self.load_buttonTab1.setFixedHeight(35)
        self.save_buttonTab1=QPushButton("Save settings")
        self.save_buttonTab1.setFixedWidth(100)
        self.save_buttonTab1.setFixedHeight(35)
        self.stop_buttonTab2=QPushButton("Stop / Reset")
        self.stop_buttonTab2.setFixedWidth(90)
        self.stop_buttonTab2.setFixedHeight(35)
        self.start_button=QPushButton("Start")
        self.start_button.setFixedWidth(50)
        self.start_button.setFixedHeight(35)
        self.quit_buttonTab2=QPushButton("Quit")
        self.quit_buttonTab2.setFixedWidth(50)
        self.quit_buttonTab2.setFixedHeight(35)
        self.load_buttonTab2=QPushButton("Load settings")
        self.load_buttonTab2.setFixedWidth(100)
        self.load_buttonTab2.setFixedHeight(35)
        self.save_buttonTab2=QPushButton("Save settings")
        self.save_buttonTab2.setFixedWidth(100)
        self.save_buttonTab2.setFixedHeight(35)

## Constitution of the ProgressBars

In [ ]:
        self.autoSweep_progressbar=QProgressBar()
        self.autoSweep_progressbar.setGeometry(200, 80, 250, 20)
        self.autoSweep_progressbar.setTextVisible(True)  # To display the percentage indicating the progression
        self.sweep_progressbar=QProgressBar()
        self.sweep_progressbar.setFixedWidth(150)
        self.sweep_progressbar.setTextVisible(True)

## Constitution of a LCDNumber (LCD screen style display area)

In [ ]:
        self.timeRemaining_lcdnumber=QLCDNumber()
        self.timeRemaining_lcdnumber.setFixedWidth(120)
        self.timeRemaining_lcdnumber.setFixedHeight(60)
        self.timeRemaining_lcdnumber.setDigitCount(16)  # Number of digits to display
        self.timeRemaining_lcdnumber.setSegmentStyle(QLCDNumber.Flat)  # To make the numbers more noticeable

## Constitution of the SpinBoxes (interface element with arrowed buttons)

In [ ]:
        self.rfvco_spinbox=QDoubleSpinBox()     # A DoubleSpinBox allows to display floating numbers
        self.rfvco_spinbox.setFixedWidth(70)
        self.rfvco_spinbox.setFixedHeight(25)
        self.rfvco_spinbox.setRange(NIR_RANGE_MIN_BOUNDARY_MHZ,NIR_RANGE_MAX_BOUNDARY_MHZ)  # Default range
        self.currentOutputFreqValue_spinbox=QDoubleSpinBox()
        self.currentOutputFreqValue_spinbox.setFixedWidth(80)
        self.currentOutputFreqValue_spinbox.setFixedHeight(25)
        self.currentOutputFreqValue_spinbox.setReadOnly(True)  # Deactivate the arrowed buttons (display only)
        self.currentOutputFreqValue_spinbox.setRange(0,8000)   # Setting a default range
        self.currentOutputFreqValue_spinbox.setDecimals(4)     # 4 decimal places

## Constitution of the Labels

In [ ]:
        # Constitution of a special font for some labels.
        # This font will be applied to headlines in particular
        font=QFont("Comic Sans MS", 10, QFont.Bold)

        # Creating the labels
        self.writePLL_label=QLabel()    # Empty label as the text will be set at the appropriate time
        self.writePLL_label.setAlignment(Qt.AlignCenter)  # Center alignement for the label
        self.bcountValueTab1_label=QLabel()
        self.bcountValueTab1_label.setFont(font)     # Setting the font of the label
        self.acountValueTab1_label=QLabel()
        self.acountValueTab1_label.setFont(font)
        self.ncountValueTab1_label=QLabel()
        self.ncountValueTab1_label.setFont(font)
        self.rcountValueTab1_label=QLabel()
        self.rcountValueTab1_label.setFont(font)
        self.bcountValueTab2_label=QLabel()
        self.bcountValueTab2_label.setFont(font)
        self.acountValueTab2_label=QLabel()
        self.acountValueTab2_label.setFont(font)
        self.ncountValueTab2_label=QLabel()
        self.ncountValueTab2_label.setFont(font)
        self.rcountValueTab2_label=QLabel()
        self.rcountValueTab2_label.setFont(font)
        self.rCounterLatchValueTab1_label=QLabel()
        self.rCounterLatchValueTab1_label.setFont(font)
        self.abCounterLatchValueTab1_label=QLabel()
        self.abCounterLatchValueTab1_label.setFont(font)
        self.functionLatchValueTab1_label=QLabel()
        self.functionLatchValueTab1_label.setFont(font)
        self.initLatchValueTab1_label=QLabel()
        self.initLatchValueTab1_label.setFont(font)
        self.RFsettings_label=QLabel("RF Settings")
        self.RFsettings_label.setFont(font)
        self.rCounterLatchTab1_label=QLabel("R Counter Latch =")
        self.abCounterLatchTab1_label=QLabel("AB Counter Latch =")
        self.functionLatchTab1_label=QLabel("Function Latch =")
        self.initLatchTab1_label=QLabel("Initialization Latch =")
        self.rCounterLatchValueTab2_label=QLabel()
        self.rCounterLatchValueTab2_label.setFont(font)
        self.abCounterLatchValueTab2_label=QLabel()
        self.abCounterLatchValueTab2_label.setFont(font)
        self.functionLatchValueTab2_label=QLabel()
        self.functionLatchValueTab2_label.setFont(font)
        self.initLatchValueTab2_label=QLabel()
        self.initLatchValueTab2_label.setFont(font)
        self.rCounterLatchTab2_label=QLabel("R Counter Latch =")
        self.abCounterLatchTab2_label=QLabel("AB Counter Latch =")
        self.functionLatchTab2_label=QLabel("Function Latch =")
        self.initLatchTab2_label=QLabel("Initialization Latch =")
        self.settings_label=QLabel("Settings")
        self.settings_label.setFont(font)
        self.CPset1_label=QLabel("Charge Pump current setting 1 :")
        self.CPset2_label=QLabel("Charge Pump current setting 2 :")
        self.CPgain_label=QLabel("Charge Pump Gain :")
        self.CPoutput_label=QLabel("Charge Pump Output :")
        self.fastlockMode_label=QLabel("FastLock Mode :")
        self.timeout_label=QLabel("Timeout (in PFD cycles):")
        self.PFDpolarity_label=QLabel("Phase Detector Polarity :")
        self.counterReset_label=QLabel("Counter Reset :")
        self.lockDetectPrecision_label=QLabel("Lock Detect Precision:")
        self.powerDown_label=QLabel("Power Down :")
        self.abpw_label=QLabel("Antibacklash Pulse Width :")
        self.muxout_label=QLabel("MUXOUT :")
        self.frequencySweep_label=QLabel("Frequency sweep")
        self.frequencySweep_label.setFont(font)
        self.startFreq_label=QLabel("Start frequency (in MHz):")
        self.stopFreq_label=QLabel("Stop frequency (in MHz):")
        self.spacingTab2_label=QLabel("Spacing (in MHz);")
        self.timeDelay_label=QLabel("Time delay (in ms):")
        self.currentOutputFreq_label=QLabel("Current output frequency (in MHz):")
        self.timeRemaining_label=QLabel("Time remaining:")
        self.sweepCompleted_label=QLabel() 

## Constitution of the ComboBoxes

In [ ]:
        self.serialPorts_combobox=QComboBox()  # Use of the lists defined previously to fill
        self.serialPorts_combobox.addItems(self.serialPorts)  # the comboboxes
        self.channel_combobox=QComboBox()
        self.channel_combobox.addItems(self.channels)
        self.prescaler_combobox=QComboBox()            
        self.prescaler_combobox.addItems(self.prescalerValues)
        self.CPsetting1_combobox=QComboBox()
        self.CPsetting1_combobox.addItems(self.CPsetting)
        self.CPsetting2_combobox=QComboBox()
        self.CPsetting2_combobox.addItems(self.CPsetting)
        self.CPgain_combobox=QComboBox()
        self.CPgain_combobox.addItems(self.CPgain)
        self.CPoutput_combobox=QComboBox()
        self.CPoutput_combobox.addItems(self.CPoutput)
        self.fastlockMode_combobox=QComboBox()
        self.fastlockMode_combobox.addItems(self.fastlockMode)
        self.timeout_combobox=QComboBox()
        self.timeout_combobox.addItems(self.timeoutCycles)
        self.PFDpolarity_combobox=QComboBox()
        self.PFDpolarity_combobox.addItems(self.PFDpolarity)
        self.counterReset_combobox=QComboBox()
        self.counterReset_combobox.addItems(self.counterReset)
        self.lockDetectPrecision_combobox=QComboBox()
        self.lockDetectPrecision_combobox.addItems(self.lockDetectPrecision)
        self.powerDown_combobox=QComboBox()
        self.powerDown_combobox.addItems(self.powerDown)
        self.abpw_combobox=QComboBox()
        self.abpw_combobox.addItems(self.abpw)
        self.muxout_combobox=QComboBox()
        self.muxout_combobox.addItems(self.muxout)

## Putting together the different elements

### Creation of two Tabs to store the elements

In [ ]:
        self.tab=QTabWidget()  # QTabWidget : contains the created tabs
        self.tab1=QWidget()    # A tab has a "QWidget" type
        self.tab2=QWidget()

#### First tab

In [ ]:
        # Adding widgets to layoutFL.
        # layoutFL = form style layout for the left part of the window in the first tab.
        # A form layout allows to create a label and to add a widget right next to it
        self.layoutFL=QFormLayout()
        self.layoutFL.addRow(self.RFsettings_label)
        self.layoutFL.addRow("Channel :", self.channel_combobox)
        self.layoutFL.addRow("Reference Frequency (in MHz):", self.rfref_line)
        self.layoutFL.addRow("Channel Spacing (in kHz):", self.spacingTab1_line)
        self.layoutFL.addRow("RF VCO Output Frequency (in MHz):", self.rfvco_spinbox)
        self.layoutFL.addRow("PFD Frequency (in kHz):", self.rfpfd_line)
        self.layoutFL.addRow("Prescaler :", self.prescaler_combobox)
        self.layoutFL.addRow("N Counter =", self.ncountValueTab1_label)
        self.layoutFL.addRow("R counter =", self.rcountValueTab1_label)
        self.layoutFL.addRow("B Counter =", self.bcountValueTab1_label)
        self.layoutFL.addRow("A Counter =", self.acountValueTab1_label)
        self.layoutFL.setVerticalSpacing(13)
        self.layoutFL.setHorizontalSpacing(13)
        
        # Adding widgets to layoutGR.
        # layoutGR = grid syle layout containing elements to display on the right side of the
        # window in the first tab.
        # A grid layout allows to select a specific row and column to position a widget.
        self.layoutGR=QGridLayout()
        self.layoutGR.setVerticalSpacing(35)
        self.layoutGR.setHorizontalSpacing(15)
        self.layoutGR.addWidget(self.settings_label, 0, 0, Qt.AlignTop)
        self.layoutGR.addWidget(self.CPset1_label, 1, 0)
        self.layoutGR.addWidget(self.CPsetting1_combobox, 1, 1)
        self.layoutGR.addWidget(self.CPset2_label, 2, 0)
        self.layoutGR.addWidget(self.CPsetting2_combobox, 2, 1)
        self.layoutGR.addWidget(self.CPgain_label, 3, 0)
        self.layoutGR.addWidget(self.CPgain_combobox, 3, 1)
        self.layoutGR.addWidget(self.CPoutput_label, 4, 0)
        self.layoutGR.addWidget(self.CPoutput_combobox, 4, 1)
        self.layoutGR.addWidget(self.fastlockMode_label, 5, 0)
        self.layoutGR.addWidget(self.fastlockMode_combobox, 5, 1)
        self.layoutGR.addWidget(self.timeout_label, 6, 0)
        self.layoutGR.addWidget(self.timeout_combobox, 6, 1)
        self.layoutGR.addWidget(self.PFDpolarity_label, 7, 0)
        self.layoutGR.addWidget(self.PFDpolarity_combobox, 7, 1)
        self.layoutGR.addWidget(self.counterReset_label, 1, 2)
        self.layoutGR.addWidget(self.counterReset_combobox, 1, 3)
        self.layoutGR.addWidget(self.lockDetectPrecision_label, 2, 2)
        self.layoutGR.addWidget(self.lockDetectPrecision_combobox, 2, 3)
        self.layoutGR.addWidget(self.powerDown_label, 3, 2)
        self.layoutGR.addWidget(self.powerDown_combobox, 3, 3)
        self.layoutGR.addWidget(self.abpw_label, 4, 2)
        self.layoutGR.addWidget(self.abpw_combobox, 4, 3)
        self.layoutGR.addWidget(self.muxout_label, 5, 2)
        self.layoutGR.addWidget(self.muxout_combobox, 5, 3)
        
        # Creation of a specific layout for 3 PushButtons (write PLL, auto sweep and stop) and 1 label
        self.button_box=QVBoxLayout()   # "Vertical disposition" layout type
        self.button_box.addStretch(1)   # To place the buttons to the far right side of the window
        self.button_box.addWidget(self.writePLL_button)
        self.button_box.addWidget(self.writePLL_label)
        self.button_box.addWidget(self.autoSweep_button)
        self.button_box.addWidget(self.stop_buttonTab1)
        self.button_box.addWidget(self.autoSweep_progressbar)
        self.stop_buttonTab1.setVisible(False)  # Will stay hidden for the time being
        self.autoSweep_progressbar.setVisible(False)
        self.button_box.setSpacing(5)  # Horizontal and vertical spacings between each element of the layout
        
        # Creation of a specific layout for 3 PushButtons (load, save and quit) for the first tab
        self.buttonsTab1_layout=QGridLayout()
        self.buttonsTab1_layout.addWidget(self.load_buttonTab1,0,0,Qt.AlignLeft)
        self.buttonsTab1_layout.addWidget(self.save_buttonTab1,1,0,Qt.AlignLeft)
        self.buttonsTab1_layout.addWidget(self.quit_buttonTab1,1,1,Qt.AlignRight)
        
        # Grid layout to display the contents of the PLL's registers in the first tab
        self.layoutLatchTab1=QGridLayout()
        self.layoutLatchTab1.addWidget(self.initLatchTab1_label, 0, 0)
        self.layoutLatchTab1.addWidget(self.initLatchValueTab1_label, 0, 1)
        self.layoutLatchTab1.addWidget(self.functionLatchTab1_label, 1, 0)
        self.layoutLatchTab1.addWidget(self.functionLatchValueTab1_label, 1, 1)
        self.layoutLatchTab1.addWidget(self.rCounterLatchTab1_label, 2, 0)
        self.layoutLatchTab1.addWidget(self.rCounterLatchValueTab1_label, 2, 1)
        self.layoutLatchTab1.addWidget(self.abCounterLatchTab1_label, 3, 0)
        self.layoutLatchTab1.addWidget(self.abCounterLatchValueTab1_label, 3, 1)
        self.layoutLatchTab1.setAlignment(Qt.AlignCenter)
        self.layoutLatchTab1.setVerticalSpacing(7)
        
        # Adding layouts layoutFL, layoutGR and button_box to layoutH1
        # in order to place them one next to the other
        self.layoutH1=QHBoxLayout()      # "Horizontal disposition" layout type
        self.layoutH1.addLayout(self.layoutFL)
        self.layoutH1.setSpacing(20)     # To space layouts between them
        self.layoutH1.addLayout(self.layoutGR)
        self.layoutH1.addStretch(1)
        self.layoutH1.addLayout(self.button_box)
        
        # Adding layouts layoutH1, layoutLatchTab1 and buttonsTab1_layout to the
        # first tab's final layout : layoutV1
        self.layoutV1=QVBoxLayout(self.tab1)  # We specify here that layoutV1 will be the first tab's main layout
        self.layoutV1.addLayout(self.layoutH1)
        self.layoutV1.addLayout(self.layoutLatchTab1)
        self.layoutV1.setSpacing(10)
        self.layoutV1.addLayout(self.buttonsTab1_layout)

        self.tab.addTab(self.tab1,"Main settings")  # Adding tab1 to the TabWidget

#### Second tab

In [ ]:
        # Adding widgets to layoutTab2.
        # layoutTab2 = grid layout containing a portion of the elements to display
        # in the second tab
        self.layoutTab2=QGridLayout()
        self.layoutTab2.setVerticalSpacing(25)
        self.layoutTab2.setHorizontalSpacing(10)
        self.layoutTab2.addWidget(self.frequencySweep_label,0,0)
        self.layoutTab2.addWidget(self.startFreq_label,1,0)
        self.layoutTab2.addWidget(self.startFreq_line,1,1)
        self.layoutTab2.addWidget(self.stopFreq_label,2,0)
        self.layoutTab2.addWidget(self.stopFreq_line,2,1)
        self.layoutTab2.addWidget(self.spacingTab2_label,3,0)
        self.layoutTab2.addWidget(self.spacingTab2_line,3,1)
        self.layoutTab2.addWidget(self.timeDelay_label,4,0)
        self.layoutTab2.addWidget(self.timeDelay_line,4,1)
        self.layoutTab2.addWidget(self.currentOutputFreq_label,5,0)
        self.layoutTab2.addWidget(self.currentOutputFreqValue_spinbox,5,1)
        self.layoutTab2.addWidget(self.sweep_progressbar,6,0)
        self.layoutTab2.addWidget(self.sweepCompleted_label,6,1)
        self.layoutTab2.addWidget(self.timeRemaining_label,7,0)
        self.layoutTab2.addWidget(self.timeRemaining_lcdnumber,7,1)
        self.layoutTab2.addWidget(self.start_button,8,0)
        self.layoutTab2.addWidget(self.stop_buttonTab2,8,1)
    
        # Adding some widgets to layoutCounters.
        # layoutCounters = form layout allowing to display the values of counters R, N, A and B
        # in the second tab
        self.layoutCounters=QFormLayout()
        self.layoutCounters.addRow("N Counter =", self.ncountValueTab2_label)
        self.layoutCounters.addRow("R counter =", self.rcountValueTab2_label)
        self.layoutCounters.addRow("B Counter =", self.bcountValueTab2_label)
        self.layoutCounters.addRow("A Counter =", self.acountValueTab2_label)
        self.layoutCounters.setSpacing(10)

        # Creation of a specific layout for 3 PushButtons (load, save and quit) to
        # display in the second tab
        self.buttonsTab2_layout=QGridLayout()
        self.buttonsTab2_layout.addWidget(self.load_buttonTab2,0,0,Qt.AlignLeft)
        self.buttonsTab2_layout.addWidget(self.save_buttonTab2,1,0,Qt.AlignLeft)
        self.buttonsTab2_layout.addWidget(self.quit_buttonTab2,1,1,Qt.AlignRight)

        # Grid layout to display the contents of the PLL's registers in the second tab
        self.layoutLatchTab2=QGridLayout()
        self.layoutLatchTab2.addWidget(self.initLatchTab2_label, 0, 0)
        self.layoutLatchTab2.addWidget(self.initLatchValueTab2_label, 0, 1)
        self.layoutLatchTab2.addWidget(self.functionLatchTab2_label, 1, 0)
        self.layoutLatchTab2.addWidget(self.functionLatchValueTab2_label, 1, 1)
        self.layoutLatchTab2.addWidget(self.rCounterLatchTab2_label, 2, 0)
        self.layoutLatchTab2.addWidget(self.rCounterLatchValueTab2_label, 2, 1)
        self.layoutLatchTab2.addWidget(self.abCounterLatchTab2_label, 3, 0)
        self.layoutLatchTab2.addWidget(self.abCounterLatchValueTab2_label, 3, 1)
        self.layoutLatchTab2.setVerticalSpacing(0)

        # Adding layouts layoutCounters and layoutLatchTab2 to layoutH2 (second tab)
        self.layoutH2=QHBoxLayout()
        self.layoutH2.addLayout(self.layoutCounters,Qt.AlignCenter)
        self.layoutH2.addLayout(self.layoutLatchTab2,Qt.AlignCenter)
        self.layoutH2.addStretch(1)
        self.layoutH2.setSpacing(10)
        self.layoutH2.setAlignment(Qt.AlignCenter)

        # Adding layouts layoutTab2, layoutH2 and buttonsTab2_layout to the
        # second tab's final layout : layoutV2
        self.layoutV2=QVBoxLayout(self.tab2)  # We specify here that layoutV2 will be the second tab's main layout
        self.layoutV2.addLayout(self.layoutTab2)
        self.layoutV2.addLayout(self.layoutH2)
        self.layoutV2.addStretch(3)
        self.layoutV2.addLayout(self.buttonsTab2_layout)
        
        self.tab.addTab(self.tab2,"Sweep")  # Adding tab2 to the TabWidget

### Final disposal

In [ ]:
        # lay = window's main layout containing only the 2 tabs
        self.lay=QVBoxLayout()
        self.lay.addWidget(self.tab)  # Adding the TabWidget (tab bar)
        
        self.setLayout(self.lay)      # We specify here that lay will be used as main layout
                                      # by the class

## Signal connections

In [ ]:
        # Each widget transmits one or more signals that are specific to it.
        # It is possible to connect these signals to another widget or to a handling function.
        
        # Example of connection to a handling function :
        # If "Write PLL" button is pressed, the program will execute the "writePLLbutton_clicked_event" function
        self.connect(self.writePLL_button, SIGNAL("clicked()"), self.writePLLbutton_clicked_event)
        
        # Example of connection to another widget :
        # As soon as a value is entered in "rfpfd_line", the same value will be written in "spacingTab1_line"
        self.connect(self.rfpfd_line, SIGNAL("textChanged(QString)"), self.spacingTab1_line.setText)
        
        self.connect(self.spacingTab1_line, SIGNAL("textChanged(QString)"), self.rfpfd_line.setText)
        self.connect(self.save_buttonTab1, SIGNAL("clicked()"), self.saveButton_clicked_event)
        self.connect(self.save_buttonTab2, SIGNAL("clicked()"), self.saveButton_clicked_event)
        self.connect(self.load_buttonTab1, SIGNAL("clicked()"), self.loadButton_clicked_event)
        self.connect(self.load_buttonTab2, SIGNAL("clicked()"), self.loadButton_clicked_event)
        self.connect(self.channel_combobox, SIGNAL("currentIndexChanged (int)"), self.channelCombobox_changed_event)
        self.connect(self.spacingTab1_line, SIGNAL("textEdited (QString)"), self.channel_spacing_changed_event)
        self.connect(self.quit_buttonTab1, SIGNAL("clicked()"), self.quit_button_clicked_event)
        self.connect(self.quit_buttonTab2, SIGNAL("clicked()"), self.quit_button_clicked_event)
        self.connect(self.autoSweep_button, SIGNAL('clicked()'), self.autoSweep_button_clicked_event)
        self.connect(self.stop_buttonTab1, SIGNAL('clicked()'), self.stop_buttonTab1_clicked_event)
        self.connect(self.start_button, SIGNAL('clicked()'), self.start_button_clicked_event)
        self.connect(self.stop_buttonTab2, SIGNAL('clicked()'), self.stop_buttonTab2_clicked_event)

        # As soon as a value change is detected in any widget of the first tab, the label
        # corresponding to the "Write PLL" button must be updated
        self.connect(self.channel_combobox, SIGNAL("currentIndexChanged (int)"), self.update_writePLL_button_label)
        self.connect(self.rfref_line, SIGNAL("textChanged(QString"), self.update_writePLL_button_label)
        self.connect(self.spacingTab1_line, SIGNAL("textChanged(QString)"), self.update_writePLL_button_label)
        self.connect(self.rfvco_spinbox, SIGNAL("valueChanged(double)"), self.update_writePLL_button_label)
        self.connect(self.rfpfd_line, SIGNAL("textChanged(QString)"), self.update_writePLL_button_label)
        self.connect(self.prescaler_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.CPsetting1_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.CPsetting2_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.CPgain_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.CPoutput_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.fastlockMode_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.timeout_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.PFDpolarity_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.counterReset_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.lockDetectPrecision_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.powerDown_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.abpw_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.muxout_combobox, SIGNAL("currentIndexChanged(int)"), self.update_writePLL_button_label)
        self.connect(self.load_buttonTab1, SIGNAL("clicked()"), self.update_writePLL_button_label)
        self.connect(self.autoSweep_button, SIGNAL("clicked()"), self.update_writePLL_button_label)

## Handling functions

### "Write PLL" button clicked handling function

In [ ]:
    # This function builds the PLL registers according to the values entered by the user
    # and when the "Write PLL" button is pressed.
    # Once the registers are built, they are sent to the PLL through the serial port.
    
    def writePLLbutton_clicked_event(self):
        self.stop_buttonTab1.setVisible(False)        # Make sure that this button is hidden
        self.autoSweep_progressbar.setVisible(False)  # Same for the ProgressBar
        
        # Check if there is serial port selected in th combo box
        if str(self.serialPorts_combobox.currentText())=='':  # If there isn't any port displayed, then an error pops-up
                QMessageBox.about(self, 'Error : serial port',\
                'No USB device seems to be connected to your computer.')
                self.timer2.stop()
                return
        else:   # Retrieving the serial port's number chosen by the user in the "serialPorts_combobox"
                serial_port="COM"+(str(self.serialPorts_combobox.currentText())[5])+":"  # "COMx:" format
        
        rfvco_mhz=self.rfvco_spinbox.value()    # Retrieving the value of the SpinBox
        rfvco_hz=int(rfvco_mhz * 1e6)
            
        rfref_mhz=self.rfref_line.text()    # Retrieving the string in the LineEdit
        if re.match("[0-9]{2,3}(?![\d.])",rfref_mhz)!=None:  # Check if the string is a 2 or 3 digits integer
            
                rfref_mhz=int(rfref_mhz)            # Conversion of the string into an integer
                
                # The integer must be within a specific range, if it isn't,
                # an error message is addressed to the user
                if (rfref_mhz<RF_REF_MIN_MHZ or rfref_mhz>RF_REF_MAX_MHZ):
                        QMessageBox.about(self, 'Error : incompatible string format',\
                        'Input for Reference Frequency can only be between 20 and 250 MHz')
                        return  # Exit the function in order not to execute all the following code
                        
                # If everything is OK, the MHz value is converted into Hz for further use
                else:
                        rfref_hz=int(rfref_mhz * 1e6)
                        
        # If the input string isn't a 2 or 3 digits integer, an error message is displayed        
        else:
                QMessageBox.about(self, 'Error : incompatible string format',\
                'Input for Reference Frequency can only be a 2 or 3 digits number')
                return

        rfpfd_khz=self.rfpfd_line.text()
        if re.match("[0-9]{1,5}(0)(?![\d.])",rfpfd_khz)!=None:   # Check whether or not the string is a 1 to 5 digits
                                                        # integer and terminated by a 0 (round number)
            
                rfpfd_khz=int(rfpfd_khz)    # Conversion of the string into an integer

                # The integer must be within a specific range, if it isn't,
                # an error message is addressed to the user
                if (rfpfd_khz < RF_PFD_MIN_KHZ or rfpfd_khz > RF_PFD_MAX_KHZ or rfpfd_khz % 10 != 0):
                        QMessageBox.about(self, 'Error : incompatible string format',\
                        'Input for PFD Frequency can only be between 40 and 100 000 kHz (round number)')
                        return

                # If everything is OK, the kHz value is converted in Hz for further use
                else:
                        rfpfd_hz=int(rfpfd_khz*1e3)
                        channelSpac=rfpfd_hz

        # If the input string isn't a 2 to 6 digits round number, an error message is displayed
        else:
                QMessageBox.about(self, 'Error : incompatible string format',\
                'Input for PFD Frequency can only be a 2 - 6 digits round number')
                return

        prescaler_currentIndex=self.prescaler_combobox.currentIndex()  # Retrieving the ComboBox's current index
        
        # According to the retrieved index, we can deduce the prescaler's value
        if prescaler_currentIndex==0:
                prescaler=8
        elif prescaler_currentIndex==1:
                prescaler=16
        elif prescaler_currentIndex==2:
                prescaler=32
        else:
                prescaler=64

        r_counter = int(rfref_hz/channelSpac)   # Calculations coming from the PLL's datasheet
        n_counter = int(rfvco_hz/rfpfd_hz)
        b_counter = int(n_counter/prescaler)
        a_counter = n_counter%prescaler
        self.rcountValueTab1_label.setText(str(r_counter))   # We write the counters' values inside
        self.ncountValueTab1_label.setText(str(n_counter))   # their respective labels
        self.bcountValueTab1_label.setText(str(b_counter))
        self.acountValueTab1_label.setText(str(a_counter))

        CPset1=self.CPsetting1_combobox.currentIndex()
        CPset2=self.CPsetting2_combobox.currentIndex()

        CPgain=self.CPgain_combobox.currentIndex()

        CPoutput=self.CPoutput_combobox.currentIndex()

        fastlock=self.fastlockMode_combobox.currentIndex()

        timer_counter=self.timeout_combobox.currentIndex()

        pfd_polarity=self.PFDpolarity_combobox.currentIndex()

        counter_reset=self.counterReset_combobox.currentIndex()

        lockDetect_precision=self.lockDetectPrecision_combobox.currentIndex()

        power_down=self.powerDown_combobox.currentIndex()
        
        # Power-down is a 2 bit value that we must split in 2 values of 1 bit each
        if power_down==0:
                power_down2=0
                power_down1=0
        elif power_down==1:
                power_down2=0
                power_down1=1
        else:
                power_down2=1
                power_down1=1

        abpw=self.abpw_combobox.currentIndex()

        muxout=self.muxout_combobox.currentIndex()

        # Concatenation of all the retrieved values to build the 24 bits register to send to the PLL.
        # Each value is put in the appropriate format (1 bit, 2 bits ...) in accordance with the datasheet
        function_latch_b="{0:02b}".format(prescaler_currentIndex)+"{0:1b}".format(power_down2)\
                          +"{0:03b}".format(CPset2)+"{0:03b}".format(CPset1)+"{0:04b}".format(timer_counter)+\
                          "{0:02b}".format(fastlock)+"{0:1b}".format(CPoutput)+"{0:1b}".format(pfd_polarity)+\
                         "{0:03b}".format(muxout)+"{0:1b}".format(power_down1)+"{0:1b}".format(counter_reset)+'10'

        # The initialization register is the same as the previous one except for the last
        # 2 bits (control bits)
        initialization_latch_b=function_latch_b[0:22]+'11'

        r_counter_latch_b='000'+"{0:1b}".format(lockDetect_precision)+'00'+"{0:02b}".format(abpw)+\
                           "{0:014b}".format(r_counter)+'00'
        
        ab_counter_latch_b='00'+"{0:1b}".format(CPgain)+"{0:013b}".format(b_counter)+"{0:06b}".format(a_counter)+'01'

        # Conversion of the binary values into hexadecimal values with 6 alphanumeric elements
        function_latch_hex='%06x' % int(function_latch_b, 2)  # 2 = base-2, binary
        initialization_latch_hex='%06x' % int(initialization_latch_b, 2)
        r_counter_latch_hex='%06x' % int(r_counter_latch_b, 2)
        ab_counter_latch_hex='%06x' % int(ab_counter_latch_b, 2)

        # We display the registers' values in their respective labels.
        # Display an "0x" in front on the value to indicate that it is an hexa value.
        # Uppercase letters for the esthetic
        self.functionLatchValueTab1_label.setText("0x"+str(function_latch_hex).upper())
        self.initLatchValueTab1_label.setText("0x"+str(initialization_latch_hex).upper())
        self.rCounterLatchValueTab1_label.setText("0x"+str(r_counter_latch_hex).upper())
        self.abCounterLatchValueTab1_label.setText("0x"+str(ab_counter_latch_hex).upper())
        
        # Call of the function that sends the registers through the serial port
        send_instructions(initialization_latch_hex, function_latch_hex, r_counter_latch_hex, ab_counter_latch_hex, serial_port)

        # If we get to this point, then it means the values entered by the user
        # have been processed. We can then display a small message in a label,
        # beneath the "Write PLL" button
        self.writePLL_label.setText("Data processed")

### "Auto Sweep" button clicked handling function

In [ ]:
    # This function is basically the same as the "writePLLbutton_clicked_event" except
    # for the first and last few lines.
    # It allows to sweep through the frequency range given by the "channel_combobox".
    # The registers are sent after each increment of the output frequency (rfvco).
    # The increment is given by the "spacingTab1_line".
    
    def autoSweep_button_clicked_event(self):
        self.update_writePLL_button_label()    # Make sure that the label beneath the "Write PLL" button is hidden
        self.stop_buttonTab1.setVisible(True)  # Display of the "Stop / Reset" button
        self.autoSweep_progressbar.setVisible(True)  # Display of the ProgressBar
        
        # Check if there is serial port selected in th combo box
        if str(self.serialPorts_combobox.currentText())=='':  # If there isn't any port displayed, then an error pops-up
                QMessageBox.about(self, 'Error : serial port',\
                'No USB device seems to be connected to your computer.')
                self.timer2.stop()
                return
        else:   # Retrieving the serial port's number chosen by the user in the "serialPorts_combobox"
                serial_port="COM"+(str(self.serialPorts_combobox.currentText())[5])+":"  # "COMx:" format
        '''''''''''''''''''''''''''''''''''''''''''
        
        rfvco_mhz=self.rfvco_spinbox.value()    # Retrieving the value of the SpinBox
        rfvco_hz=int(rfvco_mhz * 1e6)
            
        rfref_mhz=self.rfref_line.text()    # Retrieving the string in the LineEdit
        if re.match("[0-9]{2,3}(?![\d.])",rfref_mhz)!=None:  # Check if the string is a 2 or 3 digits integer
            
                rfref_mhz=int(rfref_mhz)            # Conversion of the string into an integer
                
                # The integer must be within a specific range, if it isn't,
                # an error message is addressed to the user
                if (rfref_mhz<RF_REF_MIN_MHZ or rfref_mhz>RF_REF_MAX_MHZ):
                        QMessageBox.about(self, 'Error : incompatible string format',\
                        'Input for Reference Frequency can only be between 20 and 250 MHz')
                        self.timer1.stop()  # If an error is raised, the timer must be stopped
                        return  # Exit the function in order not to execute all the following code
                        
                # If everything is OK, the MHz value is converted into Hz for further use
                else:
                        rfref_hz=int(rfref_mhz * 1e6)
                        
        # If the input string isn't a 2 or 3 digits integer, an error message is displayed        
        else:
                QMessageBox.about(self, 'Error : incompatible string format',\
                'Input for Reference Frequency can only be a 2 or 3 digits number')
                self.timer1.stop()
                return


        rfpfd_khz=self.rfpfd_line.text()
        if re.match("[0-9]{1,5}(0)(?![\d.])",rfpfd_khz)!=None:   # Check whether or not the string is a 1 to 5 digits
                                                        # integer and terminated by a 0 (round number)
            
                rfpfd_khz=int(rfpfd_khz)    # Conversion of the string into an integer

                # The integer must be within a specific range, if it isn't,
                # an error message is addressed to the user
                if (rfpfd_khz < RF_PFD_MIN_KHZ or rfpfd_khz > RF_PFD_MAX_KHZ or rfpfd_khz % 10 != 0):
                        QMessageBox.about(self, 'Error : incompatible string format',\
                        'Input for PFD Frequency can only be between 40 and 100 000 kHz (round number)')
                        self.timer1.stop()
                        return

                # If everything is OK, the kHz value is converted in Hz for further use
                else:
                        rfpfd_hz=int(rfpfd_khz*1e3)
                        channelSpac=rfpfd_hz

        # If the input string isn't a 2 to 6 digits round number, an error message is displayed
        else:
                QMessageBox.about(self, 'Error : incompatible string format',\
                'Input for PFD Frequency can only be a 2 - 6 digits round number')
                self.timer1.stop()
                return


        prescaler_currentIndex=self.prescaler_combobox.currentIndex()  # Retrieving the ComboBox's current index
        
        # According to the retrieved index, we can deduce the prescaler's value
        if prescaler_currentIndex==0:
                prescaler=8
        elif prescaler_currentIndex==1:
                prescaler=16
        elif prescaler_currentIndex==2:
                prescaler=32
        else:
                prescaler=64

        r_counter = int(rfref_hz/channelSpac)   # Calculations coming from the PLL's datasheet
        n_counter = int(rfvco_hz/rfpfd_hz)
        b_counter = int(n_counter/prescaler)
        a_counter = n_counter%prescaler
        self.rcountValueTab1_label.setText(str(r_counter))   # We write the counters' values inside
        self.ncountValueTab1_label.setText(str(n_counter))   # their respective labels
        self.bcountValueTab1_label.setText(str(b_counter))
        self.acountValueTab1_label.setText(str(a_counter))

        CPset1=self.CPsetting1_combobox.currentIndex()
        CPset2=self.CPsetting2_combobox.currentIndex()

        CPgain=self.CPgain_combobox.currentIndex()

        CPoutput=self.CPoutput_combobox.currentIndex()

        fastlock=self.fastlockMode_combobox.currentIndex()

        timer_counter=self.timeout_combobox.currentIndex()

        pfd_polarity=self.PFDpolarity_combobox.currentIndex()

        counter_reset=self.counterReset_combobox.currentIndex()

        lockDetect_precision=self.lockDetectPrecision_combobox.currentIndex()

        power_down=self.powerDown_combobox.currentIndex()
        
        # Power-down is a 2 bit value that we must split in 2 values of 1 bit each
        if power_down==0:
                power_down2=0
                power_down1=0
        elif power_down==1:
                power_down2=0
                power_down1=1
        else:
                power_down2=1
                power_down1=1

        abpw=self.abpw_combobox.currentIndex()

        muxout=self.muxout_combobox.currentIndex()

        # Concatenation of all the retrieved values to build the 24 bits register to send to the PLL.
        # Each value is put in the appropriate format (1 bit, 2 bits ...) in accordance with the datasheet
        function_latch_b="{0:02b}".format(prescaler_currentIndex)+"{0:1b}".format(power_down2)\
                          +"{0:03b}".format(CPset2)+"{0:03b}".format(CPset1)+"{0:04b}".format(timer_counter)+\
                          "{0:02b}".format(fastlock)+"{0:1b}".format(CPoutput)+"{0:1b}".format(pfd_polarity)+\
                         "{0:03b}".format(muxout)+"{0:1b}".format(power_down1)+"{0:1b}".format(counter_reset)+'10'

        # The initialization register is the same as the previous one except for the last
        # 2 bits (control bits)
        initialization_latch_b=function_latch_b[0:22]+'11'

        r_counter_latch_b='000'+"{0:1b}".format(lockDetect_precision)+'00'+"{0:02b}".format(abpw)+\
                           "{0:014b}".format(r_counter)+'00'
        
        ab_counter_latch_b='00'+"{0:1b}".format(CPgain)+"{0:013b}".format(b_counter)+"{0:06b}".format(a_counter)+'01'

        # Conversion of the binary values into hexadecimal values with 6 alphanumeric elements
        function_latch_hex='%06x' % int(function_latch_b, 2)  # 2 = base-2, binary
        initialization_latch_hex='%06x' % int(initialization_latch_b, 2)
        r_counter_latch_hex='%06x' % int(r_counter_latch_b, 2)
        ab_counter_latch_hex='%06x' % int(ab_counter_latch_b, 2)

        # We display the registers' values in their respective labels.
        # Display of "0x" in front on the value to indicate that it is an hexa value.
        # Uppercase letters for the esthetic
        self.functionLatchValueTab1_label.setText("0x"+str(function_latch_hex).upper())
        self.initLatchValueTab1_label.setText("0x"+str(initialization_latch_hex).upper())
        self.rCounterLatchValueTab1_label.setText("0x"+str(r_counter_latch_hex).upper())
        self.abCounterLatchValueTab1_label.setText("0x"+str(ab_counter_latch_hex).upper())
        
        # Call of the function that sends the registers through the serial port
        send_instructions(initialization_latch_hex, function_latch_hex, r_counter_latch_hex, ab_counter_latch_hex, serial_port)

        """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
        # When the registers have been sent once, the output frequency is incremented
        self.rfvco_spinbox.setValue(rfvco_mhz+(channelSpac*1e-6))

        # Display of the progress with the ProgressBar
        self.autoSweep_progressbar.setRange(self.rfvco_spinbox.minimum(),self.rfvco_spinbox.maximum())
        self.autoSweep_progressbar.setValue(rfvco_mhz)

        # If the SpinBox's maximum value is reached, the timer can be stopped
        if self.rfvco_spinbox.value()==self.rfvco_spinbox.maximum():
                self.timer1.stop()
                
        else:  # If not, the timer is relaunched
                self.frequency_autoSweep()
                
                
                
    # The following function creates and launches a timer that will count 2 seconds.
                
    def frequency_autoSweep(self):
        self.timer1=QTimer()  # Creation of the timer

        # As soon as the timer has finished counting, we return in the "autoSweep_button_clicked_event" function
        self.connect(self.timer1, SIGNAL("timeout()"), self.autoSweep_button_clicked_event)
        
        self.timer1.start(2 * 1000)  # The timer counts 2 seconds

### "Stop / Reset" button (first tab) clicked handling function

In [ ]:
    # This function interrupts the automatic frequency sweep process (Auto Sweep) 
    # when the user presses the "Stop / Reset" button of the first tab.
    # Also acts as a "reset" before launching a new auto sweep operation.
    
    def stop_buttonTab1_clicked_event(self):
        if self.timer1.isActive()==True:
                self.timer1.stop()  # Timer is stopped
        self.autoSweep_progressbar.reset()  # Reset of the ProgressBar
        self.rfvco_spinbox.setValue(self.rfvco_spinbox.minimum())  # Reset of the SpinBox
        
        # Reset of the labels containing the values of the counters and registers
        self.ncountValueTab1_label.setText('');
        self.rcountValueTab1_label.setText('');
        self.bcountValueTab1_label.setText('');
        self.acountValueTab1_label.setText('');

        self.initLatchValueTab1_label.setText('');
        self.functionLatchValueTab1_label.setText('');
        self.rCounterLatchValueTab1_label.setText('');
        self.abCounterLatchValueTab1_label.setText('');

### "Save settings" button clicked handling function

In [ ]:
    # This function allows the user to save the current parameters of the application
    # in a text file so that they can be loaded later. Thanks to the function, the
    # user doesn't have to remember all the settings he had made.

    def saveButton_clicked_event(self):
        # A small dialog box opens only displaying 2 push buttons and a line edit asking
        # the user to enter a file name to which the settings will be saved
        text, ok = QInputDialog.getText(self, 'Save settings', 'Enter a file name :')
        
        if ok and text != '':  # Retrieving the name entered by the user
            
                self.saveFile = str(text)+".txt"
                
        else :  # If the line edit is empty or if the user hits "cancel", an error is shown
            
                QMessageBox.about(self,'Error','Unable to save settings as no output file name has been chosen')
                return
            
        # The "with...as..." loop allows to open and then properly close the chosen text file
        with open(self.saveFile,"w") as fichier:  # "w" stands for "write" to open the file in write mode
            fichier.write(str(self.channel_combobox.currentIndex()))    # All the values are written in the file
            fichier.write("\n"+self.rfref_line.text())                  # with a line break seperating them
            fichier.write("\n"+str(self.rfvco_spinbox.value()))
            fichier.write("\n"+self.rfpfd_line.text())
            fichier.write("\n"+str(self.prescaler_combobox.currentIndex()))
            fichier.write("\n"+str(self.CPsetting1_combobox.currentIndex()))
            fichier.write("\n"+str(self.CPsetting2_combobox.currentIndex()))
            fichier.write("\n"+str(self.CPgain_combobox.currentIndex()))
            fichier.write("\n"+str(self.CPoutput_combobox.currentIndex()))
            fichier.write("\n"+str(self.fastlockMode_combobox.currentIndex()))
            fichier.write("\n"+str(self.timeout_combobox.currentIndex()))
            fichier.write("\n"+str(self.PFDpolarity_combobox.currentIndex()))
            fichier.write("\n"+str(self.counterReset_combobox.currentIndex()))
            fichier.write("\n"+str(self.lockDetectPrecision_combobox.currentIndex()))
            fichier.write("\n"+str(self.powerDown_combobox.currentIndex()))
            fichier.write("\n"+str(self.abpw_combobox.currentIndex()))
            fichier.write("\n"+str(self.muxout_combobox.currentIndex()))
            fichier.write("\n"+self.startFreq_line.text())
            fichier.write("\n"+self.stopFreq_line.text())
            fichier.write("\n"+self.spacingTab2_line.text())
            fichier.write("\n"+self.timeDelay_line.text())

        # The parameters being saved, a message can be addressed to the user
        QMessageBox.about(self, 'Success', 'Settings successfuly saved to "%s" file' %(self.saveFile))

### "Load settings" button clicked handling function

In [ ]:
    # This function allows to load the settings previously saved in a text file.
    
    def loadButton_clicked_event(self):
        self.stop_buttonTab1.setVisible(False)  # Make sure that this button is hidden
        self.autoSweep_progressbar.setVisible(False)  # Same for the ProgressBar
        
        # A small dialog box opens only displaying 2 push buttons and a line edit asking
        # the user to enter a file name to which the settings will be saved
        text, ok = QInputDialog.getText(self, 'Load settings', 'Enter a file name :')
        
        if ok and text != '':  # Retrieving the name entered by the user
            
                self.loadFile = str(text)+".txt"
                
        else :  # If the line edit is empty or if the user hits "cancel", an error is shown
            
                QMessageBox.about(self,'Error','Unable to load settings as no input file name has been chosen')
                return

        if file_is_readable(self.loadFile)==True:  # If the file exists...
                with open(self.loadFile,"r") as fichier:
                    liste=[]    # Creation of a list to store the values contained in the file
                    
                    for ligne in fichier:       # We go through each line of the file
                            
                        liste.append(ligne.rstrip('\n\r'))  # Make sure to remove the interspaces and line breaks
                            
                    self.channel_combobox.setCurrentIndex(int(liste[0]))
                    self.rfref_line.setText(liste[1])
                    self.rfvco_spinbox.setValue(float(liste[2]))
                    self.rfpfd_line.setText(liste[3])
                    self.prescaler_combobox.setCurrentIndex(int(liste[4]))
                    self.CPsetting1_combobox.setCurrentIndex(int(liste[5]))
                    self.CPsetting2_combobox.setCurrentIndex(int(liste[6]))
                    self.CPgain_combobox.setCurrentIndex(int(liste[7]))
                    self.CPoutput_combobox.setCurrentIndex(int(liste[8]))
                    self.fastlockMode_combobox.setCurrentIndex(int(liste[9]))
                    self.timeout_combobox.setCurrentIndex(int(liste[10]))
                    self.PFDpolarity_combobox.setCurrentIndex(int(liste[11]))
                    self.counterReset_combobox.setCurrentIndex(int(liste[12]))
                    self.lockDetectPrecision_combobox.setCurrentIndex(int(liste[13]))
                    self.powerDown_combobox.setCurrentIndex(int(liste[14]))
                    self.abpw_combobox.setCurrentIndex(int(liste[15]))
                    self.muxout_combobox.setCurrentIndex(int(liste[16]))
                    self.startFreq_line.setText(liste[17])
                    self.stopFreq_line.setText(liste[18])
                    self.spacingTab2_line.setText(liste[19])
                    self.timeDelay_line.setText(liste[20])
                    
                    # Make sure that the step of the RF VCO spinbox is equal to the value loaded in the Channel Spacing
                    # line edit
                    self.rfvco_spinbox.setSingleStep(int(self.spacingTab1_line.text())*1e-3)
                        
        else:   # If the file cannot be opened, an error message is addressed to the user
            
                QMessageBox.about(self,'File error','"%" doesn\'t seem to exist' %(self.loadFile))
                return

### "channel_combobox" value changed handling function

In [ ]:
    # When the user selects a frequency channel in the "channel_combobox", the range of the "rfvco_spinbox"
    # must be changed accordingly.
    
    def channelCombobox_changed_event(self):
        channelCombobox_currentIndex=self.channel_combobox.currentIndex() # Retrieving the ComboBox's current index
        
        # According to the channel chosen, the range of the "rfvco_spinbox" is modified
        if channelCombobox_currentIndex==0:
            self.rfvco_spinbox.setMinimum(float(NIR_RANGE_MIN_BOUNDARY_MHZ))
            self.rfvco_spinbox.setMaximum(float(NIR_RANGE_MAX_BOUNDARY_MHZ))
            self.rfvco_spinbox.setValue(float(NIR_RANGE_MIN_BOUNDARY_MHZ))
        elif channelCombobox_currentIndex==1:
            self.rfvco_spinbox.setMinimum(float(UV_RANGE_MIN_BOUNDARY_MHZ))
            self.rfvco_spinbox.setMaximum(float(UV_RANGE_MAX_BOUNDARY_MHZ))
            self.rfvco_spinbox.setValue(float(UV_RANGE_MIN_BOUNDARY_MHZ))
        else:
            self.rfvco_spinbox.setMinimum(float(VIS_RANGE_MIN_BOUNDARY_MHZ))
            self.rfvco_spinbox.setMaximum(float(VIS_RANGE_MAX_BOUNDARY_MHZ))
            self.rfvco_spinbox.setValue(float(VIS_RANGE_MIN_BOUNDARY_MHZ))

### "spacingTab1_line" text edited handling function

In [ ]:
    # As soon as the user enters a value in the "spacingTab1_line", the "rfvco_spinbox"
    # step must be updated accordingly.
    
    def channel_spacing_changed_event(self):
            
        # According to the channel spacing entered, the SpinBox's single
        # step is modified
        if self.spacingTab1_line.text()=='':  # If nothing is yet written in the "spacingTab1_line",
                                              # a default value is set
            self.rfvco_spinbox.setSingleStep(1)
            
        else:  # But if something is written, the step is set according to that value (must be put in MHz)
            
            self.rfvco_spinbox.setSingleStep(float(self.spacingTab1_line.text())*1e-3)

### "Write PLL" button label handling function

In [ ]:
    # When the "Write PLL" button is pressed, a label is displayed beneath it indicating that
    # the data is being processed. However, this label must be hidden when it no longer needs
    # to be displayed. That is what this function does.
    
    def update_writePLL_button_label(self):
        
        self.writePLL_label.setText('') # The label becomes an empty label
        return

### "Start" button (second tab) clicked handling function

In [ ]:
    # The "Start" button is used when the user wishes to initiate a frequency sweep in
    # the second tab by entering a start and stop frequency, a channel spacing and a
    # time delay. This allows to generate a series of consecutive frequencies.
    
    def start_button_clicked_event(self):
        self.sweep_progressbar.setVisible(True)  # Display the ProgressBar
        
        # Check if there is serial port selected in th combo box
        if str(self.serialPorts_combobox.currentText())=='':  # If there isn't any port displayed, then an error pops-up
                QMessageBox.about(self, 'Error : serial port',\
                'No USB device seems to be connected to your computer.')
                self.timer2.stop()
                return
        else:   # Retrieving the serial port's number chosen by the user in the "serialPorts_combobox"
                serial_port="COM"+(str(self.serialPorts_combobox.currentText())[5])+":"  # "COMx:" format

        startFreq_mhz=self.startFreq_line.text()  # Retrieving the different values
        stopFreq_mhz=self.stopFreq_line.text()
        spacingTab2_mhz=self.spacingTab2_line.text()
        timeDelay=self.timeDelay_line.text()
        
        # Start and stop frequencies must be a 4 digits integer
        if re.match("[0-9]{4}(?![\d.])",startFreq_mhz)!=None and re.match("[0-9]{4}(?![\d.])",stopFreq_mhz)!=None:
                startFreq_mhz=int(startFreq_mhz)  # Conversion of the string into an integer
                stopFreq_mhz=int(stopFreq_mhz)
                
                if startFreq_mhz>=1000 and startFreq_mhz<stopFreq_mhz and stopFreq_mhz>0 and stopFreq_mhz<=2400:
                        startFreq_hz=int(startFreq_mhz * 1e6)  # Convert in Hz for further use
                        stopFreq_hz=int(stopFreq_mhz * 1e6)
                        
                else:
                    
                        QMessageBox.about(self,"Input Error","Input for Start and Stop frequencies is a value range 1000 to\
                        2400 MHz and Start must be less than Stop Fequency.")
                        self.timer2.stop()  # If an error is raised, the timer must be stopped
                        return
                    
        else:  # An error message is displayed if the input value isn't a 4 digits integer
            
                QMessageBox.about(self,"Input Error","Input for Start and Stop frequencies can only be a 4 digits number")
                self.timer2.stop()
                return
            
        # The channel spacing value can be a floating number with up to 4 decimal places,
        # or a 1 to 4 digits integer
        if re.match("[0-9]{1,4}(?![\d.])", spacingTab2_mhz)!=None or 
           re.match("[0-9]{0,4}\.[0-9]{1,4}(?![\d.])",spacingTab2_mhz)!=None:
                
                spacingTab2_mhz=float(spacingTab2_mhz)
                
                if spacingTab2_mhz<(stopFreq_mhz - startFreq_mhz):
                        spacingTab2_hz=int(spacingTab2_mhz * 1e6)
                        
                else:
                        QMessageBox.about(self,"Input error","Input for Spacing must be lesser than Start - Stop frequencies\
                        and may be a floating number (4 decimal places max)")
                        self.timer2.stop()
                        return
                    
        else:
                QMessageBox.about(self,"Input Error","Input for Spacing can only be a 1 - 4 digits number or a floating value\
                with 1 to 4 decimal places")
                self.timer2.stop()
                return


        if re.match("[0-9]{1,5}(?![\d.])",timeDelay)!=None:
                timeDelay=int(timeDelay)
                if timeDelay>=0 and timeDelay<=10000:
                        timeDelay_ms=timeDelay
                else:
                        QMessageBox.about(self,"Input Error","Input for Time Delay must be between 0 and 10 000 ms")
                        self.timer2.stop()
                        return
        else:
                QMessageBox.about(self,"Input Error","Input for Time Delay can only be a 1 to 5 digits number")
                self.timer2.stop()
                return
            
        # We also need to retrieve the values entered by the user in the first tab, in order to build
        # the registers to send to the PLL.
        
        rfref_mhz=self.rfref_line.text()    # Retrieving the string in the LineEdit
        if re.match("[0-9]{2,3}(?![\d.])",rfref_mhz)!=None:  # Check if the string is a 2 or 3 digits integer
            
                rfref_mhz=int(rfref_mhz)            # Conversion of the string into an integer
                
                # The integer must be within a specific range, if it isn't,
                # an error message is addressed to the user
                if (rfref_mhz<RF_REF_MIN_MHZ or rfref_mhz>RF_REF_MAX_MHZ):
                        QMessageBox.about(self, 'Error : incompatible string format',\
                        'Input for Reference Frequency can only be between 20 and 250 MHz')
                        self.timer2.stop()  # If an error is raised, the timer must be stopped
                        return  # Exit the function in order not to execute all the following code
                        
                # If everything is OK, the MHz value is converted into Hz for further use
                else:
                        rfref_hz=int(rfref_mhz * 1e6)
                        
        # If the input string isn't a 2 or 3 digits integer, an error message is displayed        
        else:
                QMessageBox.about(self, 'Error : incompatible string format',\
                'Input for Reference Frequency can only be a 2 or 3 digits number')
                self.timer2.stop()
                return


        rfpfd_khz=self.rfpfd_line.text()
        if re.match("[0-9]{1,5}(0)(?![\d.])",rfpfd_khz)!=None:   # Check whether or not the string is a 1 to 5 digits
                                                        # integer and terminated by a 0 (round number)
            
                rfpfd_khz=int(rfpfd_khz)    # Conversion of the string into an integer

                # The integer must be within a specific range, if it isn't,
                # an error message is addressed to the user
                if (rfpfd_khz < RF_PFD_MIN_KHZ or rfpfd_khz > RF_PFD_MAX_KHZ or rfpfd_khz % 10 != 0):
                        QMessageBox.about(self, 'Error : incompatible string format',\
                        'Input for PFD Frequency can only be between 40 and 100 000 kHz (round number)')
                        self.timer2.stop()
                        return

                # If everything is OK, the kHz value is converted in Hz for further use
                else:
                        rfpfd_hz=int(rfpfd_khz*1e3)
                        channelSpac=rfpfd_hz

        # If the input string isn't a 2 to 6 digits round number, an error message is displayed
        else:
                QMessageBox.about(self, 'Error : incompatible string format',\
                'Input for PFD Frequency can only be a 2 - 6 digits round number')
                self.timer2.stop()
                return


        self.currentOutputFreqValue_spinbox.setRange(startFreq_mhz,stopFreq_mhz)

        currentOutputFreq_mhz=self.currentOutputFreqValue_spinbox.value()
        currentOutputFreq_hz=currentOutputFreq_mhz * 1e6

        prescaler_currentIndex=self.prescaler_combobox.currentIndex()  # Retrieving the ComboBox's current index
        
        # According to the retrieved index, we can deduce the prescaler's value
        if prescaler_currentIndex==0:
                prescaler=8
        elif prescaler_currentIndex==1:
                prescaler=16
        elif prescaler_currentIndex==2:
                prescaler=32
        else:
                prescaler=64

        r_counter = int(rfref_hz/channelSpac)   # Calculations coming from the PLL's datasheet
        n_counter = int(int(currentOutputFreq_hz)/rfpfd_hz)
        b_counter = int(n_counter/prescaler)
        a_counter = n_counter%prescaler
        self.rcountValueTab1_label.setText(str(r_counter))   # We write the counters' values inside
        self.ncountValueTab1_label.setText(str(n_counter))   # their respective labels
        self.bcountValueTab1_label.setText(str(b_counter))
        self.acountValueTab1_label.setText(str(a_counter))

        CPset1=self.CPsetting1_combobox.currentIndex()
        CPset2=self.CPsetting2_combobox.currentIndex()

        CPgain=self.CPgain_combobox.currentIndex()

        CPoutput=self.CPoutput_combobox.currentIndex()

        fastlock=self.fastlockMode_combobox.currentIndex()

        timer_counter=self.timeout_combobox.currentIndex()

        pfd_polarity=self.PFDpolarity_combobox.currentIndex()

        counter_reset=self.counterReset_combobox.currentIndex()

        lockDetect_precision=self.lockDetectPrecision_combobox.currentIndex()

        power_down=self.powerDown_combobox.currentIndex()
        
        # Power-down is a 2 bit value that we must split in 2 values of 1 bit each
        if power_down==0:
                power_down2=0
                power_down1=0
        elif power_down==1:
                power_down2=0
                power_down1=1
        else:
                power_down2=1
                power_down1=1

        abpw=self.abpw_combobox.currentIndex()

        muxout=self.muxout_combobox.currentIndex()

        # Concatenation of all the retrieved values to build the 24 bits register to send to the PLL.
        # Each value is put in the appropriate format (1 bit, 2 bits ...) in accordance with the datasheet
        function_latch_b="{0:02b}".format(prescaler_currentIndex)+"{0:1b}".format(power_down2)\
                          +"{0:03b}".format(CPset2)+"{0:03b}".format(CPset1)+"{0:04b}".format(timer_counter)+\
                          "{0:02b}".format(fastlock)+"{0:1b}".format(CPoutput)+"{0:1b}".format(pfd_polarity)+\
                         "{0:03b}".format(muxout)+"{0:1b}".format(power_down1)+"{0:1b}".format(counter_reset)+'10'

        # The initialization register is the same as the previous one except for the last
        # 2 bits (control bits)
        initialization_latch_b=function_latch_b[0:22]+'11'

        r_counter_latch_b='000'+"{0:1b}".format(lockDetect_precision)+'00'+"{0:02b}".format(abpw)+\
                           "{0:014b}".format(r_counter)+'00'
        
        ab_counter_latch_b='00'+"{0:1b}".format(CPgain)+"{0:013b}".format(b_counter)+"{0:06b}".format(a_counter)+'01'

        # Conversion of the binary values into hexadecimal values with 6 alphanumeric elements
        function_latch_hex='%06x' % int(function_latch_b, 2)  # 2 = base-2, binary
        initialization_latch_hex='%06x' % int(initialization_latch_b, 2)
        r_counter_latch_hex='%06x' % int(r_counter_latch_b, 2)
        ab_counter_latch_hex='%06x' % int(ab_counter_latch_b, 2)

        # We display the registers' values in their respective labels.
        # Display of "0x" in front on the value to indicate that it is an hexa value.
        # Uppercase letters for the esthetic
        self.functionLatchValueTab1_label.setText("0x"+str(function_latch_hex).upper())
        self.initLatchValueTab1_label.setText("0x"+str(initialization_latch_hex).upper())
        self.rCounterLatchValueTab1_label.setText("0x"+str(r_counter_latch_hex).upper())
        self.abCounterLatchValueTab1_label.setText("0x"+str(ab_counter_latch_hex).upper())
        
        # Call of the function that sends the registers through the serial port
        send_instructions(initialization_latch_hex, function_latch_hex, r_counter_latch_hex, ab_counter_latch_hex, serial_port)

        self.currentOutputFreqValue_spinbox.setValue(currentOutputFreq_mhz+spacingTab2_mhz)  # Incremente the output
                                                                                             # frequency
        
        # Setting the range of the ProgressBar
        self.sweep_progressbar.setRange(self.currentOutputFreqValue_spinbox.minimum(),\
                                        self.currentOutputFreqValue_spinbox.maximum())
        
        self.sweep_progressbar.setValue(self.currentOutputFreqValue_spinbox.value())  # Display the progress

        # Calculation of the remaining time before the end of the frequency sweep
        timeRemaining_ms=(((stopFreq_hz-(currentOutputFreq_hz))/spacingTab2_hz)*(timeDelay_ms+30))  # +30 to take into account
                                                                                                    # the program's execution
                                                                                                    # time
                
        if timeRemaining_ms>=0:  # As long as we don't reach 0, we keep diplaying the remaining time
                timeRemaining_s=int((timeRemaining_ms / 1e3)%60)                 # Seconds calculation
                timeRemaining_min=int(((timeRemaining_ms / 1e3)/ 60)%60)         # Minutes
                timeRemaining_h=int((((timeRemaining_ms / 1e3)/ 60) / 60)%24)    # Hours
                timeRemaining_d=int((((timeRemaining_ms / 1e3)/ 60) / 60) / 24)  # And days
                
                # Setting the format of the LCDNumber to "00 DAYS 00:00:00"
                text="{:02d}".format(timeRemaining_d)+' '+'DAYS '+"{:02d}".format(timeRemaining_h)+":"+\
                     "{:02d}".format(timeRemaining_min)+":"+"{:02d}".format(int(timeRemaining_s-(timeDelay_ms*1e-3)))
                    
                self.timeRemaining_lcdnumber.display(text)  # Display in the LCDNumber
                
        else:  # As soon as the limit is reached, the LCDNumber is reset
                text="00 DAYS 00:00:00"
                self.timeRemaining_lcdnumber.display(text)

        # Make sure to stop the timer if the frequency sweep is completed
        if self.currentOutputFreqValue_spinbox.value()>self.currentOutputFreqValue_spinbox.maximum():
                self.timer2.stop()
                self.sweepCompleted_label.setText("Sweep completed")  # Displaying a label indicating the end of the operation 
                
        else:  # If the max value isn't reached yet, the timer is relaunched
                self.frequency_sweep_timer(timeDelay_ms)
                
    
    # The following function creates and launches a timer counting for a time given
    # by the "timeDelay_line".
                
    def frequency_sweep_timer(self,delay):
        self.timer2=QTimer()

        # As soon as the timer finishes counting, the "start_button_clicked_event" function is called
        self.connect(self.timer2, SIGNAL("timeout()"), self.start_button_clicked_event)
        self.timer2.start(delay)

### "Stop / Reset" button (second tab) clicked handling function

In [ ]:
    # This function interrupts the frequency sweep process (second tab) when the user
    # presses the "Stop / Reset" button.
    # Also acts as a "reset" before launching a new frequency sweep operation.
    
    def stop_buttonTab2_clicked_event(self):
        
        if self.timer2.isActive()==True:
                self.timer2.stop()
        self.currentOutputFreqValue_spinbox.setValue(self.currentOutputFreqValue_spinbox.minimum()) # Reset of the SpinBox
        self.sweep_progressbar.reset()
        text="00 DAYS 00:00:00"
        self.timeRemaining_lcdnumber.display(text)  # The LCDNumber is reset
        self.sweepCompleted_label.setText('')  # Remove the label indicating the end of the frequency sweep
        
        # Reset of the labels containing the values of the counters and registers
        self.ncountValueTab2_label.setText('');
        self.rcountValueTab2_label.setText('');
        self.bcountValueTab2_label.setText('');
        self.acountValueTab2_label.setText('');

        self.initLatchValueTab2_label.setText('');
        self.functionLatchValueTab2_label.setText('');
        self.rCounterLatchValueTab2_label.setText('');
        self.abCounterLatchValueTab2_label.setText('');

### "Quit" button clicked handling function

In [ ]:
    # This function kills the current running application and 
    # shuts down the window.
    
    def quit_button_clicked_event(self):
        
        app.quit()      
        window.close() 

# Launch of the application and display of the main window

In [ ]:
# Check if there isn't already an application running
app = QApplication.instance()
if not app:
    app = QApplication(sys.argv)  # If not we can create one
    
# Establishment of a style sheet for the application.
# This style sheet controls the graphical characteristics of some widgets
app.setStyleSheet("QLabel {color: rgb(0,0,127);}"\
                  "QPushButton:flat {color: rgb(255,255,255); background-color: rgb(255,116,70);}"\
                  "QPushButton:pressed {color: rgb(255,255,255); background-color: rgb(255,168,156);\
                  border-radius: 6px; border: 2px solid rgb(255,136,112);}"\
                  "QPushButton:default {color: rgb(255,255,255); background-color: rgb(255,116,70);}"\
                  "QPushButton {color: rgb(255,255,255); background-color: rgb(255,116,70); font: bold 12px;\
                  border: 2px solid rgb(255,136,112);}"\
                  "QDoubleSpinBox {color: rgb(0,0,0); background-color: rgb(255,255,150);\
                   border: 2px solid rgb(255,116,70);}"\
                  "QComboBox {border: 2px solid rgb(255,116,70); border-radius: 3px; padding: 1px 18px 1px 3px;\
                  background-color: rgb(255,255,150);}"\
                  "QComboBox:on {padding-top: 3px; padding-left: 4px; background-color: rgb(255,183,0);\
                  color: rgb(255,255,255);}"\
                  "QComboBox::down-arrow:on {top: 1px; left: 1px;}"\
                  "QComboBox::down-arrow {color: red;}"
                  "QLineEdit {background-color: rgb(255,255,150); selection-color: rgb(240,77,48);\
                  selection-background-color: yellow; border: 2px solid rgb(255,116,70); border-radius: 3px;}"
                  "QLCDNumber {background-color : rgb(255,255,150); border: 2px solid rgb(255,116,70);}")

window = MainWindow()
window.setPalette(QPalette(QColor(255,255,255)))    # The window's background is white
window.setAutoFillBackground(True)                  # Set the background
window.show()  # Display the window once the application is running
app.exec_()    # Launch the application